In [1]:
#imported for system and file managing usage
import sys

import pandas as ps

#added to access the modules that placed in sub folder
sys.path.append('./modules')

#added to read special fucntions
from data import *

FUTURE_PERIOD_PREDICT = 1



In [2]:

filePath = '/home/cryptocurrency_Forcasting/dataset/Binance_BTCUSDT_d.csv'
x_train, y_train, x_validation, y_validation, x_test, y_test = readData(filePath, 20, 20, FUTURE_PERIOD_PREDICT)

          open      high       low     close  target Day of Week
1412  0.021212  0.019632  0.023373  0.018151       0    Thursday
1411  0.018151  0.017783  0.018948  0.015225       1      Friday
1410  0.015225  0.014749  0.017448  0.015748       0    Saturday
1409  0.015748  0.015177  0.020533  0.014859       0      Sunday
1408  0.014859  0.013692  0.018492  0.013695       1      Monday
...        ...       ...       ...       ...     ...         ...
5     0.485552  0.513248  0.487577  0.504903       1         NaN
4     0.504853  0.520019  0.497772  0.521215       0         NaN
3     0.521215  0.523300  0.480685  0.470232       1         NaN
2     0.470094  0.478316  0.461700  0.481811       1         NaN
1     0.481811  0.511104  0.492483  0.521832       0         NaN

[1412 rows x 6 columns]
